Cell 1: Imports and Setup

In [ ]:
import os
import logging
import pandas as pd
import numpy as np
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables from .env file
# Make sure your FMP_API_KEY is set in a .env file in the root directory
load_dotenv()

# Import custom modules
from src.data.loader import FmpDataLoader
from src.data.processor import DataProcessor
from src.optimization.cvar_optimizer import CVaROptimizer, RollingCVaROptimizer

Cell 2: Configuration

In [ ]:
# --- Parameters for the validation run ---
FMP_API_KEY = os.getenv("FMP_API_KEY")

# Using a smaller universe for the validation run to ensure feasibility with 5% max weight
VALIDATION_TICKERS = [
    "AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "TSLA", "META", "JPM", "JNJ", "V",
    "PG", "UNH", "HD", "MA", "BAC", "DIS", "ADBE", "CRM", "NFLX", "PFE",
    "XOM", "CVX", "KO", "PEP", "WMT"
]
START_DATE = "2020-01-01"
END_DATE = "2022-12-31"
BENCHMARK_TICKER = "SPY" # Using SPY as the benchmark

Cell 3: Data Loading and Processing

In [ ]:
# In a Jupyter Notebook, you can run async code directly in a cell by using 'await'
# Ensure you are running this in an environment where top-level await is supported (modern Jupyter/IPython)

loader = FmpDataLoader(api_key=FMP_API_KEY)
processor = DataProcessor()

# Fetch data
all_tickers = VALIDATION_TICKERS + [BENCHMARK_TICKER]
price_data = await loader.fetch_prices(all_tickers, start_date=START_DATE, end_date=END_DATE)

# Process data
returns_data = processor.calculate_returns(price_data)
cleaned_returns = processor.clean_data(returns_data)

# Separate asset and benchmark returns
asset_returns = cleaned_returns[VALIDATION_TICKERS]
benchmark_returns = cleaned_returns[BENCHMARK_TICKER]

logging.info("Data loaded and processed successfully.")
print("Asset Returns Head:")
print(asset_returns.head())

Cell 4: Run Rolling Backtest

In [ ]:
# Initialize optimizers
cvar_optimizer = CVaROptimizer(alpha=0.95, lasso_penalty=0.01)
rolling_optimizer = RollingCVaROptimizer(
    optimizer=cvar_optimizer,
    lookback_window=252, # 1 year lookback
    rebalance_frequency='Q' # Quarterly rebalance
)

# Run the backtest
logging.info("Starting rolling backtest...")
backtest_results = rolling_optimizer.backtest(
    returns=asset_returns,
    benchmark_returns=benchmark_returns
)
logging.info("Backtest completed.")

Cell 5: Display Results

In [ ]:
print("\\nBacktest Results (first 5 rebalances):")
print(backtest_results.head())